# Demo de Simulación de Balance - Cantidad de Inversiones

## Configuración inicial de librerías

In [2]:
# Contiene las clases Operacion, Caja y Banco.
import simula_balance as sb

# Contiene las reglas de renovación, por ejemplo,
# toda la caja a overnight.
import reglas_renovacion as rn

# Contiene las métricas, por ejemplo, el C46.
import metricas as met

# Se usa pandas sólo para desplegar claramente el gap de liquidez.
import pandas as pd

## t = 0: Configuración inicial del Banco

In [3]:
# Monto de capital
capital = 10.0

# Sobre este objeto caen los vencimientos de activos y pasivos.
# De aquí se obtiene el dinero para captaciones y colocaciones,
# toda operación, al iniciarse, tiene su correspondiente movimiento
# en la caja.
caja = sb.Caja()

# Construimos un Banco muy sencillo con una colocación bullet
# y una captación (también bullet).
# Lo primero son las tasas de mercado
mercado = {'on_col': 0.0, 'on_cap':0.0,
           'depo2':0.0312, 'depo3':0.033, 'depo4':0.0348,
           'inversion':0.027, 'colocacion':0.045}

credito = sb.Operacion(mercado['colocacion'], 0, 6, 100, {'id':'1', 'ap':'activo', 'producto':'credito'})
deposito = sb.Operacion(mercado['depo2'], 0, 2, 90, {'id':'1', 'ap':'pasivo', 'producto':'depo'})
banco = sb.Banco(caja, [credito, deposito], capital)

### Se construye y muestra el gap de liquidez inicial del banco.

In [4]:
# Se determina el gap de liquidez del Banco.
gap = met.construye_gap_liquidez(banco)

# El gap de liquidez se guarda en un dataframe para imprimirlo bien.
pd_gap = pd.DataFrame([gap()])

# Se muestra el resultado
print "Gap de Liquidez a tiempo t = ", banco.tiempo_actual()
print pd_gap
print 'c46: ', met.c46(banco)

Gap de Liquidez a tiempo t =  0
     0  1       2  3  4  5       6  7  8
0  0.0  0 -91.404  0  0  0  106.75  0  0
c46:  0


### Se muestra el estado de resultados inicial.

In [5]:
# Se calcula y muestra, no se almacena.
banco.estado_resultados()

{'activo': {'intereses_devengados': 0.0, 'intereses_recibidos': 0.0},
 'pasivo': {'intereses_devengados': 0.0, 'intereses_pagados': 0.0}}

### Se muestra el balance inicial.

In [6]:
# Se calcula y muestra, no se almacena.
banco.balance()

{'activo': {'caja': 0.0, 'credito': 100.0},
 'pasivo': {'depo': -90.0, 'sobregiro': 0.0},
 'patrimonio': {'capital_pagado': -10.0}}

Vamos a definir algunas funciones auxiliares que harán menos verboso el ejercicio:

In [7]:
# Variable que establece los plazos y porcentajes de renovación a cada plazo.
# Si tenemos x:y significa que al plazo x (medido en número de períodos)
# se hace un y% del monto necesario.
plazos = {2:100}

def muestra_gap():
    # se determina la situación de liquidez y
    gap = met.construye_gap_liquidez(banco)
    pd_gap = pd.DataFrame([gap()])

    # se muestra el resultado.
    print "Gap de Liquidez a tiempo t = ", banco.tiempo_actual()
    print pd_gap
    print 'c46: ', met.c46(banco)
    return

def muestra_resultados():
    # Calcula y muestra el estado de resultados
    er = banco.estado_resultados()
    for key in er:
        print key, er[key]
    return

def muestra_balance():
    # Calcula y muestra el balance
    bal = banco.balance()
    for key in bal:
        print key, bal[key]
    return

def muestra_operaciones(t):
    # Muestra los flujos de las operaciones del banco
    activos = banco.activos()
    print 'Activos:'
    print 'prod\t', 'plazo\t', 'monto\t', 'int_dev\t', 'flujo\t'
    for act in activos:
        print act.atributos()['producto'], act.vencimiento(), act.vencimiento() - t, act.monto(), act.interes_devengado(t), act.flujo()
        
    pasivos = banco.pasivos()
    print 'Pasivos:'
    print 'prod\t', 'plazo\t', 'monto\t', 'int_dev\t', 'flujo\t'
    for pas in pasivos:
        print pas.atributos()['producto'], pas.vencimiento(), pas.vencimiento() - t, pas.monto(), pas.interes_devengado(t), pas.flujo()

def avanza():
    # Se avanza un período en el tiempo,
    banco.avanza()

    # se determina la nueva situación de liquidez y
    # se muestra.
    muestra_gap()
    
    print
    
    # Se calcula y muestra el EERR.
    muestra_resultados()
    
    print
    
    # Se calcula y muestra el balance
    muestra_balance()
        
    print
    
    # Se muestran las operaciones
    muestra_operaciones(banco.tiempo_actual())
    return
    
def renueva_depo():
    # Se renueva el depósito
    rn.renueva_depos(banco, plazos)

    # se determina la nueva situación de liquidez y
    # se muestra
    muestra_gap()
    return

def toma_inversiones():
    # Se renueva la inversión
    rn.inversion(banco)

    # se determina la nueva situación de liquidez y
    # se muestra.
    muestra_gap()
    return

def arregla_c46():
    # Se arregla el C46,
    rn.cumple_c46(banco, plazos)

    # Se muestra gap
    muestra_gap()

## t = 1: Se avanza un período en el tiempo y se muestra la nueva situación de liquidez, el EERR, el balance y las operaciones (vigentes y vencidas).

In [8]:
# Se avanza un período en el tiempo,
avanza()

Gap de Liquidez a tiempo t =  1
     0       1  2  3  4       5  6  7  8
0  0.0 -91.404  0  0  0  106.75  0  0  0
c46:  71.404

activo {'intereses_devengados': 1.125, 'intereses_recibidos': 0.0}
pasivo {'intereses_pagados': 0.0, 'intereses_devengados': 0.702}

patrimonio {'capital_pagado': -10.0}
activo {'caja': 0.0, 'credito': 101.125}
pasivo {'depo': -90.702, 'sobregiro': 0.0}

Activos:
prod	plazo	monto	int_dev	flujo	
credito 6.0 5.0 100.0 1.125 106.75
Pasivos:
prod	plazo	monto	int_dev	flujo	
depo 2.0 1.0 90.0 0.702 91.404


### Se "arregla" el C46. Se compran inversiones financiándolas con depósitos a los plazos indicados en la variable **plazos**.

In [9]:
# Se arregla el C46.
arregla_c46()

Gap de Liquidez a tiempo t =  1
     0       1        2  3  4       5  6  7  8
0  0.0 -19.518 -72.5179  0  0  106.75  0  0  0
c46:  0


In [10]:
muestra_operaciones(1)

Activos:
prod	plazo	monto	int_dev	flujo	
credito 6.0 5.0 100.0 1.125 106.75
inversion 2.0 1.0 71.404 0.0 71.886
Pasivos:
prod	plazo	monto	int_dev	flujo	
depo 2.0 1.0 90.0 0.702 91.404
depo 3.0 2.0 71.404 0.0 72.5179


In [11]:
muestra_balance()

patrimonio {'capital_pagado': -10.0}
activo {'inversion': 71.404, 'caja': 0.0, 'credito': 101.125}
pasivo {'depo': -162.106, 'sobregiro': 0.0}


In [12]:
muestra_resultados()

activo {'intereses_devengados': 1.125, 'intereses_recibidos': 0.0}
pasivo {'intereses_pagados': 0.0, 'intereses_devengados': 0.702}


## t = 2: Se avanza nuevamente en el tiempo y se determina la nueva situación de liquidez.

In [13]:
# Se avanza un período en el tiempo,
avanza()

Gap de Liquidez a tiempo t =  2
        0        1  2  3       4  5  6  7  8
0 -19.518 -72.5179  0  0  106.75  0  0  0  0
c46:  72.0359

activo {'intereses_devengados': 2.25, 'intereses_recibidos': 0.482}
pasivo {'intereses_pagados': 1.404, 'intereses_devengados': 0.557}

patrimonio {'capital_pagado': -10.0}
activo {'inversion': 0.0, 'caja': 0.0, 'credito': 102.25}
pasivo {'depo': -71.961, 'sobregiro': -19.518}

Activos:
prod	plazo	monto	int_dev	flujo	
credito 6.0 4.0 100.0 2.25 106.75
inversion 2.0 0.0 71.404 0.0 71.886
Pasivos:
prod	plazo	monto	int_dev	flujo	
depo 2.0 0.0 90.0 0.0 91.404
depo 3.0 1.0 71.404 0.557 72.5179


In [14]:
banco._caja._movimientos

[('capital', 0, 10.0),
 ('1credito', 0, -100.0),
 ('1depo', 0, 90.0),
 ('1-invinversion', 1, -71.404),
 ('1-fin_invdepo', 1, 71.404),
 ('1depo', 2, -91.404),
 ('1-invinversion', 2, 71.886)]

### El déficit de caja se debe al vencimiento por 92.7 de depósitos comparado con 73.2453 de inversiones. Vamos a renovar el depósito y la inversión.

In [15]:
# Se renueva el depósito
renueva_depo()

Gap de Liquidez a tiempo t =  2
        0        1        2  3       4  5  6  7  8
0  71.886 -72.5179 -92.8299  0  106.75  0  0  0  0
c46:  0


In [16]:
muestra_operaciones(2)

Activos:
prod	plazo	monto	int_dev	flujo	
credito 6.0 4.0 100.0 2.25 106.75
inversion 2.0 0.0 71.404 0.0 71.886
Pasivos:
prod	plazo	monto	int_dev	flujo	
depo 2.0 0.0 90.0 0.0 91.404
depo 3.0 1.0 71.404 0.557 72.5179
depo 4.0 2.0 91.404 0.0 92.8299


In [17]:
muestra_gap()

Gap de Liquidez a tiempo t =  2
        0        1        2  3       4  5  6  7  8
0  71.886 -72.5179 -92.8299  0  106.75  0  0  0  0
c46:  0


### El C46 está Ok, pero hay que renovar la inversión.

In [18]:
# Se renueva la inversión
toma_inversiones()

Gap de Liquidez a tiempo t =  2
     0       1        2  3       4  5  6  7  8
0  0.0 -0.1467 -92.8299  0  106.75  0  0  0  0
c46:  0


In [19]:
muestra_operaciones(2)

Activos:
prod	plazo	monto	int_dev	flujo	
credito 6.0 4.0 100.0 2.25 106.75
inversion 2.0 0.0 71.404 0.0 71.886
inversion 3.0 1.0 71.886 0.0 72.3712
Pasivos:
prod	plazo	monto	int_dev	flujo	
depo 2.0 0.0 90.0 0.0 91.404
depo 3.0 1.0 71.404 0.557 72.5179
depo 4.0 2.0 91.404 0.0 92.8299


### Veamos como queda el balance.

In [19]:
# Se muestra el balance
banco.balance()

{'activo': {'caja': 0.0, 'credito': 102.25, 'inversion': 71.886},
 'pasivo': {'depo': -163.365, 'sobregiro': 0.0},
 'patrimonio': {'capital_pagado': -10.0}}

In [20]:
muestra_resultados()

activo {'intereses_devengados': 2.25, 'intereses_recibidos': 0.482}
pasivo {'intereses_pagados': 1.404, 'intereses_devengados': 0.557}


## t = 3: Se avanza nuevamente en el tiempo y se determina la nueva situación de liquidez.

In [20]:
# Se avanza un período en el tiempo
avanza()

Gap de Liquidez a tiempo t =  3
        0        1  2       3  4  5  6  7  8
0 -0.1467 -92.8299  0  106.75  0  0  0  0  0
c46:  72.9766

activo {'intereses_devengados': 3.375, 'intereses_recibidos': 0.9672000000000001}
pasivo {'intereses_pagados': 2.5179, 'intereses_devengados': 0.713}

patrimonio {'capital_pagado': -10.0}
activo {'inversion': 0.0, 'caja': 0.0, 'credito': 103.375}
pasivo {'depo': -92.117, 'sobregiro': -0.1467}

Activos:
prod	plazo	monto	int_dev	flujo	
credito 6.0 3.0 100.0 3.375 106.75
inversion 2.0 -1.0 71.404 0.0 71.886
inversion 3.0 0.0 71.886 0.0 72.3712
Pasivos:
prod	plazo	monto	int_dev	flujo	
depo 2.0 -1.0 90.0 0.0 91.404
depo 3.0 0.0 71.404 0.0 72.5179
depo 4.0 1.0 91.404 0.713 92.8299


## Veamos los movimientos de caja.

In [21]:
banco._caja._movimientos

[('capital', 0, 10.0),
 ('1credito', 0, -100.0),
 ('1depo', 0, 90.0),
 ('1-invinversion', 1, -71.404),
 ('1-fin_invdepo', 1, 71.404),
 ('1depo', 2, -91.404),
 ('1-invinversion', 2, 71.886),
 ('r1depo', 2, 91.404),
 ('inv-2-1inversion', 2, -71.886),
 ('1-fin_invdepo', 3, -72.5179),
 ('inv-2-1inversion', 3, 72.3712)]

## Se renueva el depósito.

In [22]:
# renovar depo.
renueva_depo()

Gap de Liquidez a tiempo t =  3
         0        1        2       3  4  5  6  7  8
0  72.3712 -92.8299 -73.6492  106.75  0  0  0  0  0
c46:  0.4587


In [23]:
muestra_operaciones(3)

Activos:
prod	plazo	monto	int_dev	flujo	
credito 6.0 3.0 100.0 3.375 106.75
inversion 2.0 -1.0 71.404 0.0 71.886
inversion 3.0 0.0 71.886 0.0 72.3712
Pasivos:
prod	plazo	monto	int_dev	flujo	
depo 2.0 -1.0 90.0 0.0 91.404
depo 3.0 0.0 71.404 0.0 72.5179
depo 4.0 1.0 91.404 0.713 92.8299
depo 5.0 2.0 72.5179 0.0 73.6492


### Hay que arreglar el C46, lo haremos activando la caja en inversiones.

In [25]:
# Se arregla el C46,
toma_inversiones()

Gap de Liquidez a tiempo t =  3
     0        1        2       3  4  5  6  7  8
0  0.0 -19.9702 -73.6492  106.75  0  0  0  0  0
c46:  0


In [26]:
muestra_operaciones(3)

Activos:
prod	plazo	monto	int_dev	flujo	
credito 6.0 3.0 100.0 3.375 106.75
inversion 2.0 -1.0 71.404 0.0 71.886
inversion 3.0 0.0 71.886 0.0 72.3712
inversion 4.0 1.0 72.3712 0.0 72.8597
Pasivos:
prod	plazo	monto	int_dev	flujo	
depo 2.0 -1.0 90.0 0.0 91.404
depo 3.0 0.0 71.404 0.0 72.5179
depo 4.0 1.0 91.404 0.713 92.8299
depo 5.0 2.0 72.5179 0.0 73.6492


In [27]:
arregla_c46()

Gap de Liquidez a tiempo t =  3
     0        1        2       3  4  5  6  7  8
0  0.0 -19.9702 -73.6492  106.75  0  0  0  0  0
c46:  0


In [28]:
muestra_operaciones(3)

Activos:
prod	plazo	monto	int_dev	flujo	
credito 6.0 3.0 100.0 3.375 106.75
inversion 2.0 -1.0 71.404 0.0 71.886
inversion 3.0 0.0 71.886 0.0 72.3712
inversion 4.0 1.0 72.3712 0.0 72.8597
Pasivos:
prod	plazo	monto	int_dev	flujo	
depo 2.0 -1.0 90.0 0.0 91.404
depo 3.0 0.0 71.404 0.0 72.5179
depo 4.0 1.0 91.404 0.713 92.8299
depo 5.0 2.0 72.5179 0.0 73.6492


## t = 4. Volvemos a avanzar.

In [29]:
# Se avanza un período en el tiempo,
avanza()

Gap de Liquidez a tiempo t =  4
         0        1       2  3  4  5  6  7  8
0 -19.9702 -73.6492  106.75  0  0  0  0  0  0
c46:  73.6194

activo {'intereses_devengados': 4.5, 'intereses_recibidos': 1.4557}
pasivo {'intereses_pagados': 3.9438, 'intereses_devengados': 0.5656}

patrimonio {'capital_pagado': -10.0}
activo {'inversion': 0.0, 'caja': 0.0, 'credito': 104.5}
pasivo {'depo': -73.0835, 'sobregiro': -19.9702}

Activos:
prod	plazo	monto	int_dev	flujo	
credito 6.0 2.0 100.0 4.5 106.75
inversion 2.0 -2.0 71.404 0.0 71.886
inversion 3.0 -1.0 71.886 0.0 72.3712
inversion 4.0 0.0 72.3712 0.0 72.8597
Pasivos:
prod	plazo	monto	int_dev	flujo	
depo 2.0 -2.0 90.0 0.0 91.404
depo 3.0 -1.0 71.404 0.0 72.5179
depo 4.0 0.0 91.404 0.0 92.8299
depo 5.0 1.0 72.5179 0.5656 73.6492


### Renovamos el depósito.

In [30]:
# Se renueva el depósito
renueva_depo()

Gap de Liquidez a tiempo t =  4
         0        1       2  3  4  5  6  7  8
0  72.8597 -73.6492  12.472  0  0  0  0  0  0
c46:  0


### Renovamos la inversión.

In [31]:
# Se renueva la inversión,
toma_inversiones()

Gap de Liquidez a tiempo t =  4
     0       1       2  3  4  5  6  7  8
0  0.0 -0.2977  12.472  0  0  0  0  0  0
c46:  0


## t = 5: Volvemos a avanzar.

In [32]:
# Se avanza un período en el tiempo,
avanza()

Gap de Liquidez a tiempo t =  5
        0       1  2  3  4  5  6  7  8
0 -0.2977  12.472  0  0  0  0  0  0  0
c46:  0

activo {'intereses_devengados': 5.625, 'intereses_recibidos': 1.9475}
pasivo {'intereses_pagados': 5.0751, 'intereses_devengados': 0.7241}

patrimonio {'capital_pagado': -10.0}
activo {'inversion': 0.0, 'caja': 0.0, 'credito': 105.625}
pasivo {'depo': -93.554, 'sobregiro': -0.2977}

Activos:
prod	plazo	monto	int_dev	flujo	
credito 6.0 1.0 100.0 5.625 106.75
inversion 2.0 -3.0 71.404 0.0 71.886
inversion 3.0 -2.0 71.886 0.0 72.3712
inversion 4.0 -1.0 72.3712 0.0 72.8597
inversion 5.0 0.0 72.8597 0.0 73.3515
Pasivos:
prod	plazo	monto	int_dev	flujo	
depo 2.0 -3.0 90.0 0.0 91.404
depo 3.0 -2.0 71.404 0.0 72.5179
depo 4.0 -1.0 91.404 0.0 92.8299
depo 5.0 0.0 72.5179 0.0 73.6492
depo 6.0 1.0 92.8299 0.7241 94.278


## t = 6: Volvemos a avanzar.

In [33]:
# Se avanza un período en el tiempo,
avanza()

Gap de Liquidez a tiempo t =  6
         0  1  2  3  4  5  6  7  8
0  12.1743  0  0  0  0  0  0  0  0
c46:  0

activo {'intereses_devengados': 0.0, 'intereses_recibidos': 8.6975}
pasivo {'intereses_pagados': 6.5232, 'intereses_devengados': 0.0}

patrimonio {'capital_pagado': -10.0}
activo {'inversion': 0.0, 'caja': 12.1743, 'credito': 0.0}
pasivo {'depo': -0.0, 'sobregiro': 0.0}

Activos:
prod	plazo	monto	int_dev	flujo	
credito 6.0 0.0 100.0 0.0 106.75
inversion 2.0 -4.0 71.404 0.0 71.886
inversion 3.0 -3.0 71.886 0.0 72.3712
inversion 4.0 -2.0 72.3712 0.0 72.8597
inversion 5.0 -1.0 72.8597 0.0 73.3515
Pasivos:
prod	plazo	monto	int_dev	flujo	
depo 2.0 -4.0 90.0 0.0 91.404
depo 3.0 -3.0 71.404 0.0 72.5179
depo 4.0 -2.0 91.404 0.0 92.8299
depo 5.0 -1.0 72.5179 0.0 73.6492
depo 6.0 0.0 92.8299 0.0 94.278


In [34]:
for x in banco._caja._movimientos:
    print x

('capital', 0, 10.0)
('1credito', 0, -100.0)
('1depo', 0, 90.0)
('1-invinversion', 1, -71.404)
('1-fin_invdepo', 1, 71.404)
('1depo', 2, -91.404)
('1-invinversion', 2, 71.886)
('r1depo', 2, 91.404)
('inv-2-1inversion', 2, -71.886)
('1-fin_invdepo', 3, -72.5179)
('inv-2-1inversion', 3, 72.3712)
('r1-fin_invdepo', 3, 72.5179)
('inv-3-1inversion', 3, -72.3712)
('r1depo', 4, -92.8299)
('inv-3-1inversion', 4, 72.8597)
('rr1depo', 4, 92.8299)
('inv-4-1inversion', 4, -72.8597)
('r1-fin_invdepo', 5, -73.6492)
('inv-4-1inversion', 5, 73.3515)
('1credito', 6, 106.75)
('rr1depo', 6, -94.278)


In [37]:
result_6m = 2.1743
round(8.6975-6.5232,4)

2.1743